In [25]:
import pandas as pd
import requests
import json
from transformers import pipeline

In [28]:
from google.colab import userdata
API_KEY = userdata.get('Alpha2')

In [4]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) Y
Token is valid (permission: write).
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub.
Run the following command in your terminal in 

In [45]:
import pandas as pd
import requests
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# Load FinBERT model and tokenizer
model_name = "ProsusAI/finbert"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

# Ensure the model is on the GPU for faster inference
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Function to fetch news data from Alpha Vantage
def fetch_news_data(api_key, ticker):
    url = f"https://www.alphavantage.co/query?function=NEWS_SENTIMENT&tickers={ticker}&apikey={api_key}"
    response = requests.get(url)
    return response.json()

# Function to analyze sentiment using FinBERT
def analyze_sentiment(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True).to(device)
    with torch.no_grad():
        outputs = model(**inputs)
    logits = outputs.logits
    sentiment = torch.argmax(logits, dim=-1).cpu().numpy()[0]
    labels = ["neutral", "positive", "negative"]  # FinBERT label mapping
    return labels[sentiment]

# Fetching news data for multiple tickers
tickers = ["AAPL", "TSLA", "GOOGL"]

# Initialize an empty DataFrame to store results from all tickers
all_data = pd.DataFrame()

for ticker in tickers:
    news_data = fetch_news_data(API_KEY, ticker)

    # Check if the news_data contains articles
    if "feed" in news_data and news_data["feed"]:
        articles = news_data["feed"]

        # Creating a DataFrame to store the results
        df = pd.DataFrame(articles)

        # Apply the FinBERT model to analyze sentiment for each article's title
        df['LLM_Sentiment'] = df['title'].apply(analyze_sentiment)

        # Add ticker symbol to the DataFrame
        df['Ticker'] = ticker

        # Append the DataFrame to the all_data DataFrame
        all_data = pd.concat([all_data, df], ignore_index=True)
    else:
        print(f"No news data available for {ticker}. Stopping further processing.")
        break  # Stop processing if the feed is empty

# Display the consolidated results
if not all_data.empty:
    all_data = all_data[['Ticker', 'title', 'summary', 'LLM_Sentiment', 'overall_sentiment_label']]
    display(all_data)
else:
    print("No news data available for the selected tickers.")


No news data available for GOOGL. Stopping further processing.


,Ticker,title,summary,LLM_Sentiment,overall_sentiment_label
0,AAPL,Why I Think Warren Buffett Made the Right Move...,Berkshire Hathaway sold about half of its Appl...,negative,Somewhat-Bullish
1,AAPL,Should WisdomTree U.S. Total Dividend ETF ( D...,Style Box ETF report for DTD ...,negative,Somewhat-Bullish
2,AAPL,Is iShares Core Dividend Growth ETF ( DGRO ) ...,Smart Beta ETF report for ...,neutral,Somewhat-Bullish
3,AAPL,Great News for Apple and Qualcomm Stock Investors,Smartphone sales are expected to grow much fas...,neutral,Neutral
4,AAPL,Elon Musk Questions Legality Of OpenAI Going F...,"Tesla and SpaceX CEO Elon Musk, who co-founded...",negative,Neutral
...,...,...,...,...,...
95,TSLA,Elon Musk's Original 'Secret Master Plan' Scru...,Tesla Inc. TSLA has quietly removed CEO Elon M...,negative,Somewhat-Bullish
96,TSLA,"NVDA, AFRM, CRWD, IREN, TSLA: Top 5 Trending S...","On Wednesday, major U.S. stock indices closed ...",negative,Somewhat-Bullish
97,TSLA,Lucid Air EV 'Indestructible' In New Retrofit:...,A new version of the Lucid Air Sapphire is bei...,negative,Neutral
98,TSLA,Tesla's LiDAR Mystery! Is a Model X Sporting L...,"A Tesla, Inc. TSLA Model X testing mule, spott...",negative,Neutral
